In [1]:
import argparse

import numpy as np
import pandas as pd
import scipy.sparse as sp
import torch
from sklearn.model_selection import KFold
from tqdm import tqdm

In [2]:
%load_ext autoreload
%autoreload 2

from load_data import load_data
from model import GModel
from myutils import roc_auc, translate_result
from optimizer import Optimizer
from sampler import NewSampler

In [3]:
class Args:
    def __init__(self):
        self.device = "cpu"  # cuda:number or cpu
        self.data = "ctrp"  # Dataset{gdsc or ccle}


args = Args()
res, drug_finger, exprs, mut, cna, null_mask, pos_num = load_data(args)

In [4]:
cell_sum = np.sum(res, axis=1)
drug_sum = np.sum(res, axis=0)

target_dim = [
    # 0,  # Cell
    1  # Drug
]
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [5]:
def mofgcn_new(
    exprs,
    cna,
    mut,
    drug_finger,
    res_mat,
    null_mask,
    target_dim,
    target_index,
    evaluate_fun,
    args,
    seed,
    device,
):
    sampler = NewSampler(res_mat, null_mask, target_dim, target_index, seed)
    val_labels = sampler.test_data[sampler.test_mask] 

    if len(np.unique(val_labels)) < 2:
        print(f"Target {target_index} skipped: Validation set has only one class.")
        return None, None 
    
    model = GModel(
        adj_mat=sampler.train_data,
        gene=exprs,
        cna=cna,
        mutation=mut,
        sigma=2,
        k=11,
        iterates=3,
        feature_drug=drug_finger,
        n_hid1=192,
        n_hid2=36,
        alpha=5.74,
        device=device,
    )
    opt = Optimizer(
        model,
        sampler.train_data,
        sampler.test_data,
        sampler.test_mask,
        sampler.train_mask,
        roc_auc,
        lr=5e-4,
        epochs=1000,
        device=device,
    ).to(device)
    epoch, true_data, predict_data = opt()
    return true_data, predict_data

In [6]:
n_kfold = 1
true_data_s = pd.DataFrame()
predict_data_s = pd.DataFrame()
for dim in target_dim:
    for seed, target_index in enumerate(tqdm(np.arange(res.shape[dim]))):
        if dim:
            if drug_sum[target_index] < 10:
                continue
        else:
            if cell_sum[target_index] < 10:
                continue
        epochs = []
        for fold in range(n_kfold):
            true_data, predict_data = mofgcn_new(
                exprs=exprs,
                cna=cna,
                mut=mut,
                drug_finger=drug_finger,
                res_mat=res,
                null_mask=null_mask,
                target_dim=dim,
                target_index=target_index,
                evaluate_fun=roc_auc,
                args=args,
                seed=seed,
                device=device,
            )

        true_data_s = pd.concat(
            [true_data_s, translate_result(true_data)], ignore_index=True
        )
        predict_data_s = pd.concat(
            [predict_data_s, translate_result(predict_data)], ignore_index=True
        )

  0%|          | 0/460 [00:00<?, ?it/s]

epoch:   0 loss:0.711093 auc:0.4744
epoch:  20 loss:0.391045 auc:0.4458
epoch:  40 loss:0.375316 auc:0.4244
epoch:  60 loss:0.372069 auc:0.4115
epoch:  80 loss:0.370801 auc:0.3993
epoch: 100 loss:0.369826 auc:0.3904
epoch: 120 loss:0.368474 auc:0.3729
epoch: 140 loss:0.365840 auc:0.3576


  0%|          | 1/460 [00:10<1:24:03, 10.99s/it]

epoch: 160 loss:0.360472 auc:0.3504
Fit finished.
epoch:   0 loss:0.784080 auc:0.3467
epoch:  20 loss:0.396339 auc:0.5067
epoch:  40 loss:0.377669 auc:0.4978
epoch:  60 loss:0.373083 auc:0.5156
epoch:  80 loss:0.371605 auc:0.5511
epoch: 100 loss:0.370387 auc:0.6178
epoch: 120 loss:0.368649 auc:0.6844
epoch: 140 loss:0.365435 auc:0.7111
epoch: 160 loss:0.360587 auc:0.7556
epoch: 180 loss:0.355866 auc:0.8089
epoch: 200 loss:0.352895 auc:0.8178
epoch: 220 loss:0.343606 auc:0.8311
epoch: 240 loss:0.340357 auc:0.6978
epoch: 260 loss:0.325761 auc:0.7378
epoch: 280 loss:0.317652 auc:0.7289


  0%|          | 2/460 [00:27<1:49:34, 14.35s/it]

...

epoch: 340 loss:0.305421 auc:0.7284
Fit finished.
epoch:   0 loss:0.728217 auc:0.4842
epoch:  20 loss:0.395622 auc:0.4635
epoch:  40 loss:0.375892 auc:0.4750
epoch:  60 loss:0.372337 auc:0.4840
epoch:  80 loss:0.371043 auc:0.4920
epoch: 100 loss:0.369972 auc:0.5008
epoch: 120 loss:0.368500 auc:0.5084
epoch: 140 loss:0.365734 auc:0.5139
epoch: 160 loss:0.360815 auc:0.5219
epoch: 180 loss:0.354997 auc:0.5065


100%|█████████▉| 459/460 [1:12:48<00:09,  9.49s/it]

epoch: 200 loss:0.351647 auc:0.4585
Fit finished.
epoch:   0 loss:0.817471 auc:0.3457
epoch:  20 loss:0.399250 auc:0.4078
epoch:  40 loss:0.377660 auc:0.4559
epoch:  60 loss:0.372903 auc:0.4720
epoch:  80 loss:0.371231 auc:0.4967
epoch: 100 loss:0.370101 auc:0.5082
epoch: 120 loss:0.368832 auc:0.5216
epoch: 140 loss:0.366849 auc:0.5302
epoch: 160 loss:0.363052 auc:0.5310
epoch: 180 loss:0.357752 auc:0.5086
epoch: 200 loss:0.351583 auc:0.4814
epoch: 220 loss:0.342846 auc:0.4724


100%|██████████| 460/460 [1:12:58<00:00,  9.52s/it]

epoch: 240 loss:0.334630 auc:0.4755
Fit finished.


In [7]:
true_data_s.to_csv(f"new_drug_true_{args.data}.csv")
predict_data_s.to_csv(f"new_drug_pred_{args.data}.csv")